# 用户社交数据（user_friends.csv）处理
（只取训练集和测试集中出现的用户ID）

数据来源于Kaggle竞赛：Event Recommendation Engine Challenge，根据
events they’ve responded to in the past
user demographic information
what events they’ve seen and clicked on in our app
用户对某个事件是否感兴趣

竞赛官网：
https://www.kaggle.com/c/event-recommendation-engine-challenge/data

user_friends.csv文件：共2维特征
user：用户ID
friends：以空格隔开的用户好友ID列表

# 导入工具包

In [1]:
import pandas as pd

import numpy as np
import scipy.sparse as ss
import scipy.io as sio

#保存数据
import pickle

from sklearn.preprocessing import normalize

总的用户数目超过训练集和测试集中的用户，
为节省处理时间和内存，先去处理train和test，得到竞赛需要用到的事件和用户
然后对在训练集和测试集中出现过的事件和用户建立新的ID索引
先运行user_event.ipynb,
得到事件列表文件：PE_userIndex.pkl

# 读取之前算好的测试集和训练集中出现过的用户

In [2]:
#读取训练集和测试集中出现过的事件列表
userIndex = pickle.load(open("PE_userIndex.pkl", 'rb'))
n_users = len(userIndex)

print("number of users in train & test :%d" % n_users)

number of users in train & test :3391


# 读取之前用户-事件分数矩阵，将朋友参加事件的影响扩展到用户

In [3]:
#用户-事件关系矩阵
userEventScores = sio.mmread("PE_userEventScores")

#后续用于将用户朋友参加的活动影响到用户
eventsForUser = pickle.load(open("PE_eventsForUser.pkl", 'rb'))

# user_friends.csv

In [4]:
#读取数据
#不能被read_csv正确解码，放弃，自己写文件读取和数据解析函数
"""
  找出某用户的那些朋友，想法非常简单
  1)如果你有更多的朋友，可能你性格外向，更容易参加各种活动
  2)如果你朋友会参加某个活动，可能你也会跟随去参加一下
"""
 
#用户有多少个朋友
numFriends = np.zeros((n_users))
userFriends = ss.dok_matrix((n_users, n_users))
    
fin = open("user_friends.csv", 'rb')
#字段：user，friends
fin.readline()                # skip header

#ln = 0
for line in fin:  #对每个用户
    #if ln % 2000 == 0:  #打印进展
        #print "Loading line: ", ln
        
    cols = line.strip().decode().split(",")
    user = str(cols[0])    #user
    
    if user in userIndex:   #该用户在训练集和测试集的用户列表中
        friends = cols[1].split(" ")  #friends
        i = userIndex[user]       #该用户的索引
        numFriends[i] = len(friends)
        for friend in friends:  #该用户的每个朋友
            str_friend = str(friend)
            if str_friend in userIndex:  #如果朋友也在训练集或测试集中出现
                j = userIndex[str_friend]   #朋友的索引
            
                # the objective of this score is to infer the degree to
                # and direction in which this friend will influence the
                # user's decision, so we sum the user/event score for
                # this user across all training events.
            
                #userEventScores为用户对事件的打分（interested - not interseted）
                #在Users-Events.ipynb中计算好了
                eventsForUser = userEventScores.getrow(j).todense()
            
                #所有朋友参加活动的数量（平均频率）
                score = eventsForUser.sum() / np.shape(eventsForUser)[1]
                userFriends[i, j] += score
                userFriends[j, i] += score
            
    #ln += 1
fin.close()
    

#用户的朋友数目
# 归一化数组
sumNumFriends = numFriends.sum(axis=0)
numFriends = numFriends / sumNumFriends
sio.mmwrite("UF_numFriends", np.matrix(numFriends))

#
userFriends = normalize(userFriends, norm="l1", axis=0, copy=False)
sio.mmwrite("UF_userFriends", userFriends)


In [5]:
numFriends

array([  1.00767089e-04,   7.23593462e-06,   2.62101632e-04, ...,
         2.90241377e-04,   1.72054445e-04,   9.27271621e-05])